In [15]:
import pandas as pd
import gender_guesser.detector as gender

In [147]:
from os import listdir
from os.path import isfile, join

mypath='/home/lafuente/Documents/Symbolic Music/giant-midi/midis'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [152]:
directory=pd.DataFrame(data={'path':onlyfiles})

directory['fullname']=directory['path'].apply(lambda x: x.split(',')[1]+' '+x.split(',')[0])

directory['composer_gender']=directory['path'].apply(lambda x: x.split(',')[1].split(' ')[1]).apply(str.capitalize).map(lambda x: gd.get_gender(x))

females_list=['Lucrecia Roces Kasilag','Dosia McKay','Akiana Molina','Leokadiya Kashperova','Henriette-Caroline Mennechet de Barival']

directory['composer_gender']=directory['composer_gender'].apply(lambda gender: 'female' if gender=='mostly_female' or (gender in females_list) else gender)


In [151]:
#df=pd.read_csv('../maestro-v3.0.0/maestro-v3.0.0.csv')

df=pd.read_csv('full_music_pieces_youtube_similarity_pianosoloprob_split.csv')

df['fullname']=df['firstname']+' '+df['surname']

In [203]:
directory=directory[directory['composer_gender'].isin(['male','female'])]

df=directory

In [205]:
# ##=============== keep only rows with similarity values====================##
# def convert_to_int_or_leave_as_string(x):
#     try:
#         return float(x)
#     except ValueError:
#         return x

# # Apply the function to the 'similarity' column
# df['similarity'] = df['similarity'].apply(convert_to_int_or_leave_as_string)

# df=df[df['similarity'].apply(lambda x: isinstance(x, float))]

# gd = gender.Detector()

# df['composer_gender']=df['firstname'].apply(lambda x: x.split(' ')[0]).apply(str.capitalize).map(lambda x: gd.get_gender(x))



##========== get number of female songs ====================================##
n_female_songs=df[df['composer_gender']=='female'].shape[0]

female_composers=df[df['composer_gender']=='female']

#female_composers['similarity']=female_composers['similarity'].astype(float)


##========== get fairly balanced male sample ================================##
male_composers=df[df['composer_gender']=='male']

unique_fullnames = male_composers['fullname'].unique()


sampled_fullnames = pd.Series(unique_fullnames).sample(n=female_composers['fullname'].nunique(), random_state=42).tolist()


filtered_male_composers = male_composers[male_composers['fullname'].isin(sampled_fullnames)]


if len(filtered_male_composers) > n_female_songs:   
    sampled_male_composers = filtered_male_composers.sample(n=n_female_songs, random_state=42)
else:   
    sampled_male_composers = filtered_male_composers


##============== concatenate into a single df ================================##
giant_midi_data=pd.concat([sampled_male_composers,female_composers])

#giant_midi_data=giant_midi_data[['fullname','surname', 'firstname', 'music', 'nationality', 'birth', 'death','similarity','composer_gender']]

In [207]:
##============== analysis ================================##
grouped=giant_midi_data[['fullname','path']].groupby('fullname').count().reset_index()

grouped=grouped.merge(giant_midi_data[['fullname','composer_gender']].drop_duplicates(),how='left',on='fullname')

grouped=grouped.sort_values(by='path',ascending=False)

import plotly.express as px

fig=px.bar(grouped,x='fullname',y='path',color='composer_gender')
fig.show()

In [213]:
giant_midi_data['name']=giant_midi_data['path'].apply(lambda x: x.split(',')[-2])

In [214]:
giant_midi_data.to_csv('./dataframes/giant_midi_scores.csv',index=False)